In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from imutils import paths
import pandas as pd
import numpy as np
import pickle
import random
import os
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report





# Variable Constants #
ORIG_INPUT_DATASET = "mask-classifier-images"
BASE_PATH = "dataset"

TRAIN = "training"
TEST = "evaluation"
VAL = "validation"

CLASSES = ["Mask", "No_Mask"]

LE_PATH = os.path.sep.join(["output", "le.cpickle"])
BASE_CSV_PATH = "output"

MODEL_PATH = os.path.sep.join(["output", "model.cpickle"])

BATCH_SIZE = 32


In [ ]:
from tensorflow.ekars.preprocessing.image import 